# VRS workflow

## Setup Dependencies

In [ ]:
!pip install seqrepo ga4gh.vrs[extras]==2.0.0a2 ga4gh.vrs
%pip install --upgrade --no-cache-dir terra-notebook-utils

In [ ]:
from ga4gh.vrs.extras.vcf_annotation import VCFAnnotator
from pathlib import Path
from terra_notebook_utils import drs, table

import datetime
import multiprocessing
import os 
import pickle
import requests
import subprocess

In [ ]:
# store relevant variables

%env SEQREPO_ROOT=/home/jupyter/seqrepo
%env VCFTOOLS_DIR=/home/jupyter/vcftools
%env PERL5LIB=/home/jupyter/vcftools/src/perl/
%env VCFTOOLS=/home/jupyter/vcftools/src/cpp/vcftools
%env OUTPUT=/home/jupyter/output
%env SPLIT_DIR=/home/jupyter/split
!mkdir $SPLIT_DIR
!mkdir $OUTPUT

In [ ]:
!bash ~/setup.sh

## Get VCFs ([GREGoR](https://anvil.terra.bio/#workspaces/anvil-datastorage/AnVIL_GREGoR_UW_CRDR_U05_GRU/data); [1000G](https://explore.anvilproject.org/files?filter=%5B%7B%22categoryKey%22%3A%22files.file_format%22%2C%22value%22%3A%5B%22.vcf%22%5D%7D%5D))

In [ ]:
! mkdir ~/vcf

# # get gatk_exome (local only) TODO
# os.environ["GATK_VCF_FILE"] = "gatk_exome.vcf"
# os.environ["GATK_VCF"] = "gs://fc-secure-ff61133c-8d50-49c3-84e1-a93489621c7f/gatk_exome.vcf"
# # TODO: check if already exists
# ! gsutil -u $GOOGLE_PROJECT cp $GATK_VCF "~/vcf/$GATK_VCF_FILE"

# # for GREGoR
# ! gsutil -u $GOOGLE_PROJECT cp 'gs://fc-secure-9cc1ea56-e4bf-47fe-9229-40d4d7f452bf/1369747.merged.matefixed.sorted.markeddups.recal.g.vcf.gz' ~/vcf/1369747.merged.matefixed.sorted.markeddups.recal.g.vcf.gz
# ! gunzip ~/vcf/1369747.merged.matefixed.sorted.markeddups.recal.g.vcf.gz

In [ ]:
def get_vcf_from_drs_uri(uri):
    file_name = drs.info(uri)["name"]
    vcf_path = f"/home/jupyter/vcf/{file_name}"
    
    print("trying...")
    
    if not os.path.exists(vcf_path):
        drs.copy(uri, vcf_path)
        print(f"written to {vcf_path} \n")
    else:
        print(f"file already exists at {vcf_path} \n")


    return vcf_path

drs_uris = [
#             "drs://drs.anv0:v2_1dc37127-8329-3ce5-b019-4ed290946fcb", # 1000G_omni2.5.chrY.vcf
#             "drs://drs.anv0:v2_7f4758a6-7b61-3290-98a0-585e12af3997", # 1000G_phase1.snps.high_confidence.hg38.chrY.vcf
#             "drs://drs.anv0:v2_f33f8839-b3e9-39e2-8318-a650b995cc45", # HG002vCHM13_20200921_mm2_PBCCS_sniffles.s2l20.refined.nSVtypes.ism.vcf
            "drs://drs.anv0:v2_950a3291-674c-32e7-8466-7dfd40ddd7a8", # HG006vCHM13_20200921_mm2_PBCCS_sniffles.s2l20.refined.nSVtypes.ism.vcf
#             "drs://drs.anv0:v2_497867a4-c4b4-3dc0-955b-4c638331e0aa", # HG006vGRCh38_mm2_ONT_sniffles.s2l20.refined.nSVtypes.ism.vcf
#             "drs://drs.anv0:v2_16d0715f-d70f-3180-a022-f40e6f1d1ce4", # HG02080vCHM13_20200921_mm2_ONT_sniffles.s2l20.refined.nSVtypes.ism.vcf
#             "drs://drs.anv0:v2_02513ada-a6bd-3be6-887f-f1f85ed4649f", # Homo_sapiens_assembly38.dbsnp138.chrY.vcf
#             "drs://drs.anv0:v2_6f73895f-4d17-3a3d-a7bb-5f4e2f972f55", # chm13_hifi_HG007.crossaligner.vcf
#             "drs://drs.anv0:v2_c6545454-c695-3e73-8b08-29757fca9fd5", # grch38_HG002_trio_merged.vcf
#            "drs://drs.anv0:v2_4dd732ef-973b-354a-823a-c48b4c9f4a1f" # grch38_hifi_HG006.crossaligner.vcf
           ]
drs_vcfs = [get_vcf_from_drs_uri(uri) for uri in drs_uris]

In [ ]:
# get file sizes

def human_size(num_bytes):
    suffixes = ['B', 'KB', 'MB', 'GB', 'TB', 'PB', 'EB', 'ZB', 'YB']
    for suffix in suffixes:
        if num_bytes < 1024:
            return f"{num_bytes:.2f} {suffix}" 
        num_bytes /= 1024

    return f"{num_bytes:.2f} {suffixes[-1]}"

for uri in drs_uris:
    info = drs.info(uri)
    print(f"{info['name']}: {human_size(info['size'])}")

## Split before annotate

In [ ]:
!echo $VCFTOOLS

In [ ]:
!ls ~/

In [ ]:
!echo $VCFTOOLS_DIR

In [ ]:
vcf_path = drs_vcfs[0]
# %env VCF_PATH = vcf_path

# os.environ['VCF_PATH'] = "~/vcf/"+os.environ['GATK_VCF_FILE']

# w/ chr prefix
# os.environ["VCF_PATH"] = vcf_path
# ! (seq 1 22; echo X; echo Y) | xargs -P 0 -I PATH $VCFTOOLS --recode --vcf $VCF_PATH --chr PATH --out ~/split/chrPATH

split_vcf_cmd = f"(seq 1 22; echo X; echo Y) | \
               xargs -P 0 -I PATH $VCFTOOLS --recode --vcf {vcf_path} \
               --chr chrPATH --out $SPLIT_DIR/chrPATH"

subprocess.run(split_vcf_cmd, shell=True, check=True) 

# no chr prefix
# ! (seq 1 22; echo X; echo Y) | xargs -P 0 -I PATH ~/vcftools-vcftools-d511f46/src/cpp/vcftools --recode --vcf $VCF_PATH --chr PATH --out ~/split/chrPATH

In [ ]:
# TODO: parse logs to get outputs on how many were filtered out

In [ ]:
ls -l $SPLIT_DIR/*.recode.vcf | wc -l

In [ ]:
# annotate each of them
# TODO: fix the outputs coming from this

! ls -1 $SPLIT_DIR/*.recode.vcf |  xargs -P 0 -I PATH python3 -m ga4gh.vrs.extras.vcf_annotation --vcf_in PATH --vcf_out PATH.vcf.gz --vrs_pickle_out PATH.pkl --seqrepo_root_dir ~/seqrepo/latest/

# # GREGoR
# !python3 -m ga4gh.vrs.extras.vcf_annotation --vcf_in 1369747.merged.matefixed.sorted.markeddups.recal.g.vcf  --vcf_out 1369747.merged.matefixed.sorted.markeddups.recal.g.vcf.output.vcf.gz --vrs_pickle_out 1369747.merged.matefixed.sorted.markeddups.recal.g.vcf.vrs_objects.pkl  --seqrepo_root_dir ~/seqrepo/latest/

In [ ]:
!ls -l $SPLIT_DIR/*.vcf.vcf.gz | wc -l
!ls -l $SPLIT_DIR/*.vcf.pkl | wc -l

# assert (!ls -l ~/split/*.vcf.vcf.gz | wc -l) == 24, "incorrect number of output vcf.gz files created"
# assert (!ls -l ~/split/*.vcf.pkl | wc -l) == 24, "incorrect number of outputted pickle files"

In [ ]:
# join the files
!ls -1 $SPLIT_DIR/*.vcf.vcf.gz | xargs $PERL5LIB/vcf-concat > $OUTPUT/merged_output.vcf

In [ ]:
!ls ~/output/

In [ ]:
# TODO: remove the pair of them

### Random python annotate

In [ ]:
import logging

logger = logging.getLogger("ga4gh.vrs.extras.vcf_annotation")
logger.setLevel(level=logging.INFO)

# create annotated vcf test file 
def annotate_vcf(path):
    '''param stem: path of input vcf file'''
    stem = path.replace(".vcf", "")
    
    input_vcf = path
    output_vcf = f"{stem}.output.vcf.gz"
    output_pkl = f"{stem}-vrs-objects.pkl"

    
    vcf_annotator = VCFAnnotator(seqrepo_root_dir="/home/jupyter/seqrepo/latest")
    vcf_annotator.annotate(vcf_in=input_vcf, vcf_out=output_vcf, vrs_pickle_out=output_pkl)
    # vcf_annotator.annotate(vcf_in=input_vcf, vrs_pickle_out=output_pkl)
    
# annotate_vcf("/home/jupyter/split", "chr1.recode")
successes = set()
for vcf_path in drs_vcfs:
    try:
        print("trying...", vcf_path)
        annotate_vcf(vcf_path)
        print("worked \n")
        successes.add(vcf_path)
    except Exception as e:
        print(e)
        print("unsucessful, see logs above \n")

print(f"total successes: {len(successes)}/{len(drs_vcfs)} \nList...")
for vcf_path in drs_vcfs:
    print(f"{vcf_path}: {'✓' if vcf_path in successes else 'x'}")

In [ ]:
# annotate w vrs id asking for output vcf

import logging

logger = logging.getLogger("ga4gh.vrs.extras.vcf_annotation")
logger.setLevel(level=logging.INFO)

# create annotated vcf test file 
def annotate_vcf(path):
    '''param stem: path of input vcf file'''
    stem = path.replace(".vcf", "")
    
    input_vcf = path
    output_vcf = f"{stem}.output.vcf.gz"
    output_pkl = f"{stem}-vrs-objects.pkl"

    
    vcf_annotator = VCFAnnotator(seqrepo_root_dir="/home/jupyter/seqrepo/latest")
    vcf_annotator.annotate(vcf_in=input_vcf, vcf_out=output_vcf, vrs_pickle_out=output_pkl)
    # vcf_annotator.annotate(vcf_in=input_vcf, vrs_pickle_out=output_pkl)
    
# annotate_vcf("/home/jupyter/split", "chr1.recode")
successes = set()
for vcf_path in drs_vcfs:
    try:
        print("trying...", vcf_path)
        annotate_vcf(vcf_path)
        print("worked \n")
        successes.add(vcf_path)
    except Exception as e:
        print(e)
        print("unsucessful, see logs above \n")

print(f"total successes: {len(successes)}/{len(drs_vcfs)} \nList...")
for vcf_path in drs_vcfs:
    print(f"{vcf_path}: {'✓' if vcf_path in successes else 'x'}")

In [ ]:
for vcf_path in drs_vcfs:
    if "HG02080vCHM13_20200921" in vcf_path:
        print(vcf_path)
    else:
        continue
#     if "chm13_hifi_HG007" in vcf_path:
#         print("trying...", vcf_path)
#         annotate_vcf(vcf_path)
#         print("worked \n")
    try:
        print("trying...", vcf_path)
        annotate_vcf(vcf_path)
        print("worked \n")
        successes.add(vcf_path)
    except Exception as e:
        print(e)
        print("unsucessful, see logs above \n")

In [ ]:
# annotate w vrs id only pickle outputted

logger = logging.getLogger("ga4gh.vrs.extras.vcf_annotation")
# logger.setLevel(level=logging.ERROR)
logger.disabled = True

# create annotated vcf test file 
def annotate_vcf_pkl_only(path):
    '''param stem: path of input vcf file'''
    stem = path.replace(".vcf", "")
    
    input_vcf = path
    output_vcf = f"{stem}.output.vcf.gz"
    output_pkl = f"{stem}-vrs-objects.pkl"

    
    vcf_annotator = VCFAnnotator(seqrepo_root_dir="/home/jupyter/seqrepo/latest")
    vcf_annotator.annotate(vcf_in=input_vcf, vrs_pickle_out=output_pkl)
    
successes = set()
for i, vcf_path in enumerate(drs_vcfs):
    print("starting... \n")
    # annotate to output pkl
    try:
        print("trying...", vcf_path)
        annotate_vcf_pkl_only(vcf_path)
        print("worked \n")
        successes.add(vcf_path)
    except Exception as e:
        print(e)
        print("unsucessful, see logs above \n")
    
    # get pickle totals
    try:
        with open(output_pkl, 'rb') as f:
            vrs_objects = pickle.load(f)

        # get total num_variants
        vcf_reader = vcf.Reader(open(vcf_path, 'r'))
        num_variants = sum(1 for record in vcf_reader)

        # view details
        print(f'num_vrs_objects to num_varaints: {len(vrs_objects)}/{num_variants}={(len(vrs_objects)/num_variants):.2f}%')
    except:
        print("unable to get pickle totals, file may not exist")
    print()

print(f"total successes: {len(successes)}/{len(drs_vcfs)} \nList...")
for vcf_path in drs_vcfs:
    print(f"{vcf_path}: {'✓' if vcf_path in successes else 'x'}")

In [ ]:
import vcf


# for input_vcf_file in ["/home/jupyter/vcf/long_read_sv_jasmine_Trios_IndividualCallsets_CHM13_HG005_Trio_HG006vCHM13_20200921_mm2_PBCCS_sniffles.s2l20.refined.nSVtypes.ism.vcf"]:
for input_vcf_file in ["/home/jupyter/vcf/long_read_minimap2_alignments_HG02080vCHM13_20200921_mm2_ONT_sniffles.s2l20.refined.nSVtypes.ism.vcf"]:
    output_vcf_file = "/home/jupyter/vcf/long_read.test.vcf"

    vcf_reader = vcf.Reader(open(input_vcf_file, 'r'))
    vcf_writer = vcf.Writer(open(output_vcf_file, 'w'), vcf_reader)

    for record in vcf_reader:
        record.INFO['VRS_ALLELE_ID'] = 'ga4gh:VA.xksahgfowdfdwofd,ga4gh:VA.xksahgfowdfdwofd'
        vcf_writer.write_record(record)

vcf_writer.close()

### show loaded files

In [ ]:
# from pprint import pprint
# import pickle
# import ast
# import requests
# import datetime

# # log progress
# progress_interval = 50000

# # load pickled dict
# with open(output_pkl, 'rb') as f:
#     print(datetime.datetime.now().isoformat(), 'opened pickle')
#     vrs_objects = pickle.load(f)
#     c = 0
#     for k, v in vrs_objects.items():
#         vrs_objects[k] = ast.literal_eval(v)
#         c += 1
#         if c % progress_interval == 0:
#             print(datetime.datetime.now().isoformat(), c)

# # view details        
# print('number of vrs objects', len(vrs_objects))

In [ ]:
pickle_paths = !ls -1 ~/split/*.vcf.pkl
pickle_paths

In [ ]:
# get percent of loaded variants
import vcf
import pickle
import ast

# load pickled dict
# for vcf_path in drs_vcfs:

def unpickle_generator(file_name):
    """Unpickle vrs objects, yields (key,vrs_object)"""
    with open(file_name, 'rb') as f:
        vrs_objects = pickle.load(f)
        for k, v in vrs_objects.items():
            yield k, ast.literal_eval(v)
            
def unpickle(file_name):
    """Unpickle vrs objects to single dict"""
    with open(file_name, 'rb') as f:
        vrs_objects = pickle.load(f)
        for k, v in vrs_objects.items():
            vrs_objects[k] = ast.literal_eval(v)
    
    return vrs_objects

vrs_dicts = []

for path in pickle_paths:
    vrs_dict = unpickle(path)
    vrs_dicts.append(vrs_dict)

    # get total num_variants
    # TODO: reference the new merged file bc some might have been filtered out
    vcf_reader = vcf.Reader(open(path[:-4], 'r'))
    num_variants = sum(1 for record in vcf_reader)

#     num_vrs_objs = sum((1 for _ in vrs_objects))
    num_vrs_objs = len(vrs_dict)

    # view details
    
    print(path.split("/")[-1], end=" ")
    if num_variants == 0: 
        print(f"no variants") 
    else:
        print(f'vrs_objects:variants = {num_vrs_objs}/{num_variants} = {(50*num_vrs_objs/num_variants):.1f}%')

# TODO on combining: have to think about this more bc large files will have to be held in memory

In [ ]:
import pysam

for vcf_path in drs_vcfs:
    # get pickle totals
    output_pkl = vcf_path.replace(".vcf", "")+"-vrs-objects.pkl"
    with open(output_pkl, 'rb') as f:
        vrs_objects = pickle.load(f)

    # get total num_variants
    vcf_reader = pysam.VariantFile(open(vcf_path, 'r'))
    num_variants = sum(1 for record in vcf_reader)
    print(f'num_vrs_objects to num_variants: {len(vrs_objects)}/{num_variants}={(100*len(vrs_objects)/num_variants):.2f}%')
    print()

In [ ]:
# def unpickle(file_name):
#     """Unpickle vrs objects, yields (key,vrs_object)"""
#     with open(file_name, 'rb') as f:
#         vrs_objects = pickle.load(f)
#         for k, v in vrs_objects.items():
#             yield k, ast.literal_eval(v)
            
            
            
# chr1_vrs_objects = unpickle(output_pkl)
# print('number of vrs objects', list(chr1_vrs_objects))

# Query remote services

## MetaKB (cancervariants.org)

In [ ]:
def meta_kb(item: tuple):
    """Query metakb"""
    k, _ = item
    
    response = requests.get(f"http://metakb-dev-eb.us-east-2.elasticbeanstalk.com/api/v2/search/studies?variation={_['id']}&detail=false")
    response_json = response.json()
    
    if response_json['warnings'] == []:
        summary = {}
        summary['description'] = response_json['statements'][0]['description']
        return (k, _['_id'], summary)

def meta_kb_old(item: tuple):
    """Query metakb"""
    k, _ = item
    
    response = requests.get(f"https://dev-search.cancervariants.org/api/v2/search?variation={_['id']}&detail=false")
    response_json = response.json()
    
    if response_json['warnings'] == []:
        summary = {}
        summary['description'] = response_json['statements'][0]['description']
        return (k, _['_id'], summary)

In [ ]:
print("elastic beanstalk link")
for vrs_dict in vrs_dicts:
    hits = []
    
    for obj in vrs_dict.items():
        potential_hit = meta_kb(obj)
        if potential_hit:
            hits.append(potential_hit)
    
    if len(vrs_dict) == 0:
        continue

    hit_rate = len(hits)/len(vrs_dict)    
    print(f"hit rate of VRS IDs: {len(hits)}/{len(vrs_dict)}={100*hit_rate:.1f}%" )
    if len(hits) > 0:
        print("first few hits")
        print(hits[:3])

In [ ]:
# old link

for vrs_dict in vrs_dicts:
    hits = []
    
    for obj in vrs_dict.items():
        potential_hit = meta_kb_old(obj)
        if potential_hit:
            hits.append(potential_hit)
    
    if len(vrs_dict) == 0:
        print("no vrs_ids")
        continue

    hit_rate = len(hits)/len(vrs_dict)    
    print(f"hit rate of VRS IDs: {len(hits)}/{len(vrs_dict)}={100*hit_rate:.1f}%" )
    if len(hits) > 0:
        print("first few hits")
        print(hits[:3])

In [ ]:
for vrs_dict in vrs_dicts[:1]:
    hits = []
    
    for i, (k, v) in enumerate(vrs_dict.items()):
        print(v)
        if i > 10: break

In [ ]:
def meta_kb_by_sequence(item: tuple):
    """Query metakb"""
    k, _ = item
    

    response = requests.get(f"https://dev-search.cancervariants.org/api/v2/search?variation={}&detail=false")
    response_json = response.json()
    
    print(response_json['warnings'])
    if response_json['warnings'] == []:
        summary = {}
        summary['description'] = response_json['statements'][0]['description']
        return (k, _['_id'], summary)

print("trying old link")
for vrs_dict in vrs_dicts:
    hits = []
    
    for obj in vrs_dict.items():
        potential_hit = meta_kb_by_sequence(obj)
        if potential_hit:
            hits.append(potential_hit)
    
    if len(vrs_dict) == 0:
        print("no variants, skipping...")
        continue

    hit_rate = len(hits)/len(vrs_dict)    
    print(f"hit rate of VRS IDs: {len(hits)}/{len(vrs_dict)}={100*hit_rate:.1f}%" )
    if len(hits) > 0:
        print("first few hits")
        print(hits[:3])

In [ ]:
def decorate(vrs_decorator, vrs_objects, limit=20):
    """harvest data from service"""

    # log progress
    progress_interval = 1000


    # number of workers
    worker_count = 12

    with multiprocessing.Pool(worker_count) as pool:
        # call the function for each item in parallel
        c = 0
        print(datetime.datetime.now().isoformat(), c)
        for result in pool.imap(vrs_decorator, vrs_objects.items()):
            c += 1
            if result:
                print(result[0], result[-1])
            if c == limit:
                break
            if c % progress_interval == 0:
                print(datetime.datetime.now().isoformat(), c)

    print(datetime.datetime.now().isoformat(), c)
            

In [ ]:
decorate(vrs_decorator=meta_kb, vrs_objects=metakb_vrs_objects)

## ClinGen (clinicalgenome.org)

In [ ]:
def clingen(item: tuple):
    """Query clingen (old version of normalizer)"""
    k, _ = item
    _id = _['_id'].split(':')[-1].split('.')[-1]
    response = requests.get(f"https://reg.genome.network/vrs-map/digest/vrs/{_id}")
    response_json = response.json()
    if response_json['status']['code'] == 200:
        iri_response = requests.get(response_json['data']['iri'])
        iri_response_json = iri_response.json()
        return (k, _['_id'], {'communityStandardTitle': iri_response_json['communityStandardTitle']})


> Note: at this time, there is a schema mismatch between vrs-python, metakb and clingen. We will use known identifiers. Normally the annotated identifiers from the variants of interest (vcf) would be used

In [ ]:
decorate(vrs_decorator=clingen, vrs_objects=clingen_vrs_objects)